## Instalação:

Para a instalar a biblioteca do Selenium e Webdriver Manager abra o prompt de comando e execute o comando:


```
pip install --upgrade selenium
```

```
pip install webdriver_manager
```

Documentação selenium: https://selenium-python.readthedocs.io/index.html

Sempre verifique a versão do Chrome instalado na sua máquina em: ajuda > Sobre o Google Chrome. <br>
Baixar a versão do webdriver correspondente a que você esteja utilizando no Chrome:<br>
https://chromedriver.chromium.org/downloads

In [1]:
# Importando as bibliotecas 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from selenium.webdriver.common.by import By # classe BY

# Abrindo navegador e selecionando chromedriver.exe na pasta local do computador
navegador = webdriver.Chrome('C:\Program Files\Google\Chrome\chromedriver.exe')

# Abrindo o site desejado - Mega Sena
link = ('https://loterias.caixa.gov.br/Paginas/Mega-Sena.aspx')
navegador.get(link)

C:\Users\fdgce\AppData\Local\Temp\ipykernel_11332\3058110497.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navegador = webdriver.Chrome('C:\Program Files\Google\Chrome\chromedriver.exe')


A classe By serve para que possamos indentificar o elemento dentro do HTML. 
Existem várias formas de encontrar o elemento:
```
ID = "id"
NAME = "name"
XPATH = "xpath"
LINK_TEXT = "link text"
PARTIAL_LINK_TEXT = "partial link text"
TAG_NAME = "tag name"
CLASS_NAME = "class name"
CSS_SELECTOR = "css selector"
```

Exemplos:
```
find_element(By.ID, "id")
find_element(By.NAME, "name")
find_element(By.XPATH, "xpath")
find_element(By.LINK_TEXT, "link text")
find_element(By.PARTIAL_LINK_TEXT, "partial link text")
find_element(By.TAG_NAME, "tag name")
find_element(By.CLASS_NAME, "class name")
find_element(By.CSS_SELECTOR, "css selector")
```

In [2]:
# Pegando os campos do HTML pelo Xpath

# Informações sobre concurso e data
info_concurso = navegador.find_element(By.XPATH, '//*[@id="wp_resultados"]/div[1]/div/h2/span').text

# Pegando as dezenas
numeros = [] 

for i in range(6):
    i = i+1
    numeros.append(navegador.find_element(By.XPATH, f'//*[@id="ulDezenas"]/li[{i}]').text)

In [3]:
concurso = info_concurso[9:13]
data = info_concurso[-11:-1]

In [4]:
print(info_concurso)
print("Dezenas:", numeros)
print("Concurso:", concurso)
print("Data:", data)


Concurso 2543 (26/11/2022)
Dezenas: ['02', '05', '27', '30', '46', '53']
Concurso: 2543
Data: 26/11/2022


Agora que temos as informações sobre concurso e as dezenas, vamos criar nosso Data Frame:

In [5]:
sorteios = pd.DataFrame(columns = ["Data", "Concurso", "Num 1", "Num 2", "Num 3", "Num 4", "Num 5", "Num 6"])
sorteios

,Data,Concurso,Num 1,Num 2,Num 3,Num 4,Num 5,Num 6


Agora que conseguimos manipular o HTML para navegar e encontrar o sorteio que quermos, vamos fazer um loop para buscar desde o início:

In [6]:
info_concurso = navegador.find_element(By.XPATH, '//*[@id="wp_resultados"]/div[1]/div/h2/span').text
concurso = int(info_concurso[9:13])
inicio_contador = concurso

# Pegando as dezenas
numeros = [] 


for j in range(inicio_contador):
    j = j + 1
    navegador.find_element(By.XPATH, '//*[@id="buscaConcurso"]').click()
    navegador.find_element(By.XPATH, '//*[@id="buscaConcurso"]').clear()
    navegador.find_element(By.XPATH, '//*[@id="buscaConcurso"]').send_keys(j)
    navegador.find_element(By.XPATH, '//*[@id="buscaConcurso"]').send_keys(Keys.ENTER)
    data = info_concurso[-11:-1]

    # Até 0 9º sorteio len = 23
    if len(info_concurso) == 23:
        concurso = int(info_concurso[9])

        # Até 0 99º sorteio len = 24
    elif len(info_concurso) == 24:
        concurso = int(info_concurso[9:11])
    
    # Até 0 99º sorteio len = 25
    elif len(info_concurso) == 25:
        concurso = int(info_concurso[9:12])
    
    # Acima de 999 len = 26
    elif len(info_concurso[9:13]):
         concurso = int(info_concurso[9:13])

    for i in range(6):
        i = i+1
        numeros.append(navegador.find_element(By.XPATH, f'//*[@id="ulDezenas"]/li[{i}]').text)
    
    sorteios.loc[len(sorteios)] = [data, concurso, numeros[0], numeros[1], numeros[2], numeros[3], numeros[4], numeros[5]]



IndexError: list index out of range